In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

d:\anaconda\envs\OPENAI\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
words = open("names.txt" , 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i , s in enumerate(chars)}
stoi['.'] = 0
itos = {i : s for s , i in stoi.items()}
vocab_size = len(itos)
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [4]:
import random

block_size = 3
def build_dataset(words):
    X , Y = [] , []

    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            # print(''.join(itos[i] for i in context) , '--->' , itos[ix])
            context = context[1 :] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape , Y.shape) 
    return X , Y

random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr , Ytr = build_dataset(words[:n1])     #train data
Xdev , Ydev = build_dataset(words[n1:n2]) #dev data
Xte , Yte = build_dataset(words[n2:])     #test data

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [5]:
#comparing manual gradient to pyTorch gradient
def cmp(s , dt , t):
    ex = torch.all(dt == t.grad).item()       #逐元素比较是否相等，返回布尔张量，torch.all比较是否所有元素都为True
    app = torch.allclose(dt , t.grad)         #两个张量是否在允许的误差范围内
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approx: {str(app):5s} | maxdiff: {maxdiff}')

In [6]:
n_emb = 10
n_hidden = 64

g = torch.Generator().manual_seed(2147483647)
#当embcat @ W1 + b1的值较大时，tanh趋近于平坦，梯度很小，故影响损失的传递（压缩了梯度）
#神经元可能无法被激活，可能因为初始化或者优化过程使神经元死亡从而无法被激活
#所以希望初始化的预激活值更加趋近于0
C = torch.randn((vocab_size , n_emb) , generator = g)
#此处初始化方法为kaiming_noraml
W1 = torch.randn((n_emb * block_size , n_hidden) , generator = g) * (5/3) * ((n_emb * block_size) ** 0.5)
b1 = torch.randn(n_hidden , generator = g) * 0.1

#希望刚开始的输出趋近于平均分布
W2 = torch.randn((n_hidden , vocab_size) , generator = g) * 0.1
b2 = torch.randn(vocab_size , generator=g) * 0.1

bngain = torch.ones((1 , n_hidden)) * 0.1 + 1.0
bnbias = torch.ones((1 , n_hidden)) * 0.1
# bnmean_running = torch.ones((1 , n_hidden))
# bnstd_running = torch.ones((1 , n_hidden))

parameters = [C , W1 , b1 , W2 , b2 , bngain , bnbias]
total_parameters = sum(p.nelement() for p in parameters)
print(total_parameters)

for p in parameters:
    p.requires_grad = True

4137


In [7]:
batch_size = 32
n = batch_size
ix = torch.randint(0 , Xtr.shape[0] , (batch_size , ) , generator=g)
Xb , Yb = Xtr[ix] , Ytr[ix]
print(Xb.shape , Yb.shape)

torch.Size([32, 3]) torch.Size([32])


In [8]:
emb = C[Xb]
print(emb.shape)
embcat = emb.view(emb.shape[0] , -1)
print(embcat.shape)

hprebn = embcat @ W1  + b1 
#BatchNorm Layer
bnmeani = 1/n * hprebn.sum(0 , keepdim=True)  #计算均值
bndiff = hprebn - bnmeani     #中心化
bndiff2 = bndiff ** 2           
bnvar = 1 / (n - 1) * (bndiff2).sum(0 , keepdim = True)       #计算方差
bnvar_inv = (bnvar + 1e-5) ** -0.5      # 1 / sqrt(方差)   标准差的倒数
bnraw = bndiff * bnvar_inv              #标准化
hpreact = bngain * bnraw + bnbias          

h = torch.tanh(hpreact)

logits = h @ W2 + b2

#cross entropy
logit_maxes = logits.max(1 , keepdim=True).values
norm_logits = logits - logit_maxes     #稳定数值
counts = norm_logits.exp()
counts_sum = counts.sum(1 , keepdims=True)
coutns_sum_inv = counts_sum ** -1
probs = counts * coutns_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n) , Yb].mean()  #在logprobs中选择Yb对应的对数似然值

for p in parameters:
    p.grad = None

for t in [logprobs , probs , counts , counts_sum , coutns_sum_inv , 
          norm_logits , logit_maxes , logits , h , hpreact , bnraw,
          bnvar_inv , bnvar , bndiff2 , bndiff , hprebn , bnmeani , embcat , emb]:
    t.retain_grad()

loss.backward()
loss

torch.Size([32, 3, 10])
torch.Size([32, 30])


tensor(3.3603, grad_fn=<NegBackward0>)

In [9]:
# logprobs.shape
# dlogprobs = torch.zeros([32 , 27])
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n) , Yb] = -1.0/n
cmp('logprobs' , dlogprobs , logprobs)    #手动计算 = 自动计算

dprobs = dlogprobs * (1.0 / probs) 
cmp('probs' , dprobs , probs)

dcounts_sum_inv = (dprobs * counts).sum(1 , keepdims=True)
cmp('counts_sum_inv' , dcounts_sum_inv , coutns_sum_inv)

dcounts_sum = dcounts_sum_inv * (-1.0) * counts_sum ** -2
cmp('counts_sum' , dcounts_sum , counts_sum)

dcounts = dprobs * coutns_sum_inv + dcounts_sum * 1
cmp('counts' , dcounts , counts)

dnorm_logits = dcounts * counts
cmp('norm_logits' , dnorm_logits , norm_logits)

dlogit_maxes = (-1 * dnorm_logits).sum(1 , keepdims=True)
cmp('logit_maxes' , dlogit_maxes , logit_maxes)

#一种方法
# dlogits = torch.zeros_like(logits)
# _ , max_indices = logits.max(1 , keepdim=True)
# dlogits.scatter_(1 , index = max_indices , value = 1.0)    #将max所在位置置为1.0
# dlogits *= dlogit_maxes
# dlogits += dnorm_logits

#方法二    利用one-hot编码实现
dlogits = dnorm_logits.clone()
dlogits += F.one_hot(logits.max(1).indices , num_classes=logits.shape[1]) * dlogit_maxes
cmp('logits' , dlogits , logits)

#dh = 32 * 64
dh = dlogits @ W2.T
cmp('h' , dh , h)
  
#dW2 = 64 * 27
dW2 = h.T @ dlogits
cmp("W2" , dW2 , W2)

db2 = dlogits.sum(0)
cmp('b2' , db2 , b2)

dhpreact = (1.0 - h ** 2) * dh
cmp("hpreact" , dhpreact , hpreact)

dbngain = (dhpreact * bnraw).sum(0 , keepdims=True)
cmp("bngain" , dbngain , bngain)

dbnraw = dhpreact * bngain
cmp("bnraw" , dbnraw , bnraw)

dbnbias = dhpreact.sum(0 , keepdim=True)
cmp("bnbias" , dbnbias , bnbias)

dbnvar_inv = (dbnraw * bndiff).sum(0 , keepdims=True)
cmp("bndiff" , dbnvar_inv , bnvar_inv)

dbnvar = dbnvar_inv * (-0.5) * (bnvar + 1e-5) ** -1.5
cmp("bnvar" , dbnvar , bnvar)

dbndiff2 = torch.ones_like(bndiff) * 1.0 / (n - 1) * dbnvar
cmp("bndiff2" , dbndiff2 , bndiff2)

dbndiff = dbnraw * bnvar_inv + 2 * bndiff * dbndiff2
cmp("bndiff" , dbndiff , bndiff)

dbnmeani = (-1.0 * dbndiff).sum(0 , keepdims=True)
cmp("bnmeani" , dbnmeani , bnmeani)

dhprebn = dbndiff + (1.0 / n) * dbnmeani
cmp("hprebn" , dhprebn , hprebn)

dembcat = dhprebn @ W1.T
cmp("embcat" , dembcat , embcat)

dW1 = embcat.T @ dhprebn
cmp("W1" , dW1 , W1)

db1 = dhprebn.sum(0)
cmp("b1" , db1 , b1)

demb = dembcat.view(emb.shape)
cmp("emb" , demb , emb)

dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[k , j]
        dC[ix] += demb[k , j]

cmp("C" , dC , C)


logprobs        | exact: True  | approx: True  | maxdiff: 0.0
probs           | exact: True  | approx: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approx: True  | maxdiff: 0.0
counts_sum      | exact: True  | approx: True  | maxdiff: 0.0
counts          | exact: True  | approx: True  | maxdiff: 0.0
norm_logits     | exact: True  | approx: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approx: True  | maxdiff: 0.0
logits          | exact: True  | approx: True  | maxdiff: 0.0
h               | exact: True  | approx: True  | maxdiff: 0.0
W2              | exact: True  | approx: True  | maxdiff: 0.0
b2              | exact: True  | approx: True  | maxdiff: 0.0
hpreact         | exact: True  | approx: True  | maxdiff: 0.0
bngain          | exact: True  | approx: True  | maxdiff: 0.0
bnraw           | exact: True  | approx: True  | maxdiff: 0.0
bnbias          | exact: True  | approx: True  | maxdiff: 0.0
bndiff          | exact: True  | approx: True  | maxdiff: 0.0
bnvar   

In [10]:
print(counts.shape)
print(dprobs.shape)
print(coutns_sum_inv.shape)
print(counts_sum.shape)
print(norm_logits.shape)
print(logits.shape)
print(dlogits.shape)
# print(F.one_hot(logits.max(1).indices , num_classes=logits.shape[1]))
print(hpreact.shape)
print(bnraw.shape)
print(bnbias.shape)

print(bndiff.shape)
print(hprebn.shape)
print(bnmeani.shape)
print(bnvar.shape)
print(bnvar_inv.shape)

print(dhprebn.sum(0).shape)
print(dhprebn.sum(0 , keepdim=True).shape)

torch.Size([32, 27])
torch.Size([32, 27])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 27])
torch.Size([32, 27])
torch.Size([32, 27])
torch.Size([32, 64])
torch.Size([32, 64])
torch.Size([1, 64])
torch.Size([32, 64])
torch.Size([32, 64])
torch.Size([1, 64])
torch.Size([1, 64])
torch.Size([1, 64])
torch.Size([64])
torch.Size([1, 64])


In [11]:
#简单实现 （数学推导）
loss_fast = F.cross_entropy(logits , Yb)
print(loss_fast.item() , 'diff: ' , (loss_fast - loss).item())

dlogits = F.softmax(logits , dim=1)
dlogits[range(n) , Yb] -= 1
dlogits /= n

cmp('logits' , dlogits , logits)

3.36032772064209 diff:  2.384185791015625e-07
logits          | exact: False | approx: True  | maxdiff: 5.3551048040390015e-09


In [12]:
dhprebn = bngain * bnvar_inv / n * (n * dhpreact - dhpreact.sum(0) - n / (n - 1) * bnraw * (dhpreact * bnraw).sum(0))
cmp("hprebn" , dhprebn , hprebn)

hprebn          | exact: False | approx: True  | maxdiff: 2.9103830456733704e-11
